In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To install dependencies

Note: After installation, please restart the runtime if it is required.

In [2]:
!pip install av
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=9d11d14ac4704f062a948afd76febd559b6afec5e446e3d88f21a57c5cdc23ec
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


The Player & ball tracking with norfair

In [3]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import av
import time
from PIL import Image
import numpy as np
import cv2

from pb_detector import PBDetector
from pb_tracker import PBTracker

input_ = av.open("videos/test1_clip.mp4")
output = av.open("output.mp4", "w")

court_region = [[690, 330], [1900, 400], [1900, 1000], [0, 800]]
detector = PBDetector(court_region)
tracker = PBTracker()

# Make an output video stream using the input video stream as a template
in_video_stream = input_.streams.video[0]
out_video_stream = output.add_stream(codec_name="h264",
                                      width=in_video_stream.width,
                                      height=in_video_stream.height,
                                      rate=int(in_video_stream.average_rate),
                                      bit_rate=in_video_stream.bit_rate,
                                      pix_fmt=in_video_stream.pix_fmt)

# Make an output audio stream using the input audio stream as a template
in_audio_stream = input_.streams.audio[0]
out_audio_stream = output.add_stream(template=in_audio_stream)

frame_idx = 0
det_cycle = 2

st = time.time()
prec_time = st
decoding_time = 0
inference_time = 0
mot_time = 0
encoder_time = 0

for packet in input_.demux():
    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        continue

    if packet.stream == in_video_stream:
        for frame in packet.decode():
            # To get an input frame
            image = frame.to_image() # PIL image
            rgb_image = np.array(image)
            image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
            current_time = time.time()
            decoding_time += (current_time - prec_time)
            prec_time = current_time

            ##############  To process by AI engine  ############

            # To process the input frame
            player_detections = []
            ball_detections = []
            if frame_idx % det_cycle == 0:
                player_detections, ball_detections = detector.detect(image)
                current_time = time.time()
                inference_time += (current_time - prec_time)
                prec_time = current_time

            player_detections, ball_detections = tracker.run(image, player_detections, ball_detections, period=det_cycle)
            print('Frame {}: detected {} players and {} balls'.format(frame_idx, len(player_detections), len(ball_detections)))
            frame_idx += 1

            # To draw the result
            image = Image.fromarray(rgb_image)
            image = tracker.draw_detections(image, player_detections, ball_detections)
            current_time = time.time()
            mot_time += (current_time - prec_time)
            prec_time = current_time

            ####################################################

            output_frame = av.VideoFrame.from_image(image)
            output_frame.pts = None
            for output_packet in out_video_stream.encode(output_frame):
                # Write the packets to the container
                output_packet.stream = out_video_stream
                output.mux(output_packet)
    elif packet.stream == in_audio_stream:
        packet.stream = out_audio_stream
        output.mux(packet)
        current_time = time.time()
        encoder_time += (current_time - prec_time)
        prec_time = current_time

elapsed = time.time() - st

input_.close()
output.close()

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

Frame 0: detected 0 players and 0 balls
Frame 1: detected 0 players and 0 balls
Frame 2: detected 8 players and 0 balls
Frame 3: detected 8 players and 0 balls
Frame 4: detected 8 players and 0 balls
Frame 5: detected 8 players and 0 balls
Frame 6: detected 8 players and 0 balls
Frame 7: detected 8 players and 0 balls
Frame 8: detected 9 players and 0 balls
Frame 9: detected 9 players and 0 balls
Frame 10: detected 9 players and 0 balls
Frame 11: detected 8 players and 0 balls
Frame 12: detected 8 players and 0 balls
Frame 13: detected 8 players and 0 balls
Frame 14: detected 8 players and 0 balls
Frame 15: detected 8 players and 0 balls
Frame 16: detected 8 players and 0 balls
Frame 17: detected 8 players and 0 balls
Frame 18: detected 8 players and 0 balls
Frame 19: detected 8 players and 0 balls
Frame 20: detected 8 players and 0 balls
Frame 21: detected 8 players and 0 balls
Frame 22: detected 8 players and 1 balls
Frame 23: detected 8 players and 1 balls
Frame 24: detected 8 playe

In [4]:
# To print the FPS
print('Processing FPS: ', int(in_video_stream.frames / elapsed))
print('Decoding time: ', int(decoding_time))
print('Inference time: ', int(inference_time))
print('MOT time: ', int(mot_time))
print('Encoder time: ', int(encoder_time))

Processing FPS:  3
Decoding time:  124
Inference time:  85
MOT time:  76
Encoder time:  210
